In [30]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,date
import time

In [31]:

def construct_download_url(ticker, interval, period_start, period_end = str(date.today())):
	"""
    :ticker: Coin Name at yahoo finance
	:period1 & period2: 'yyyy-mm-dd' : Time Start and End
	:interval: {daily; weekly, monthly} Interval as .d,.mo,.wk
	"""

	def convert_to_seconds(period):
        
		datetime_value = datetime.strptime(period, '%Y-%m-%d')
		total_seconds = int(time.mktime(datetime_value.timetuple())) + 86400
		return total_seconds
	try:
		interval_reference = {'daily': '1d', 'weekly': '1wk', 'monthly': '1mo'}
		_interval = interval_reference.get(interval)
		if _interval is None:
			print('interval code is incorrect')
			return
		p1 = convert_to_seconds(period_start)
		p2 = convert_to_seconds(period_end)
		url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={p1}&period2={p2}&interval={_interval}&filter=history'
		return url
	except Exception as e:
		print(e)
		return

# retrive dataset
query_url = construct_download_url(ticker='BTC-USD', interval='daily', period_start='2015-05-15' )
df = pd.read_csv(query_url)
df.set_index('Date', inplace=True)

# save dataset as a CSV
df.to_csv('Bitcoin Data.csv')


In [32]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [33]:
target = df.Close
inputs = df.drop(['Close', 'Adj Close', 'Volume'],axis=1)


In [42]:
inputs.shape

(2711, 3)

In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
md = LinearRegression()

In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(inputs,target,test_size=.1)

In [36]:
md.fit(X_train,Y_train)

LinearRegression()

In [37]:
prediction = md.predict(X_test)

In [38]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test, prediction)

139298.69000782302

In [39]:
md.coef_

array([-0.55357984,  0.91279058,  0.6355968 ])

In [40]:
import pickle
with open('Mymodel.pkl','wb') as f:
    pickle.dump(md,f)

In [41]:
import pickle
with open('Mymodel.pkl','rb') as f:
    md = pickle.load(f)